# Laboratorio 6

- Irving Acosta 22781
- Ricardo Chuy 221007

[Repositorio](https://github.com/wwIrvingww/Lab6_DS.git)

## Avances 
### Carga de datos

### Librerias

In [9]:
from pathlib import Path
import json
import pandas as pd

In [10]:
# -------- helpers --------
def get_id_str(d, base="id"):
    """Devuelve d[f"{base}_str"] si existe; si no, usa d[base] -> str."""
    if not isinstance(d, dict):
        return None
    if f"{base}_str" in d and d[f"{base}_str"] is not None:
        return str(d[f"{base}_str"])
    v = d.get(base)
    if v is None:
        return None
    try:
        if isinstance(v, float) and v.is_integer():
            return str(int(v))
    except Exception:
        pass
    return str(v)

def flatten_tweet(t):
    u = t.get("user", {}) if isinstance(t.get("user"), dict) else {}

    # mentionedUsers
    musers = t.get("mentionedUsers", [])
    if not isinstance(musers, list):
        musers = []
    mentioned_ids = []
    mentioned_usernames = []
    for mu in musers:
        if isinstance(mu, dict):
            mid = get_id_str(mu, "id")
            if mid: mentioned_ids.append(mid)
            if mu.get("username"): mentioned_usernames.append(mu["username"])

    # quotedTweet
    qt = t.get("quotedTweet", {}) if isinstance(t.get("quotedTweet"), dict) else None
    qt_user = qt.get("user", {}) if qt and isinstance(qt.get("user"), dict) else {}

    # media counts
    media = t.get("media", {}) if isinstance(t.get("media"), dict) else {}
    photos_cnt = len(media.get("photos", [])) if isinstance(media.get("photos"), list) else 0
    videos_cnt = len(media.get("videos", [])) if isinstance(media.get("videos"), list) else 0
    animated_cnt = len(media.get("animated", [])) if isinstance(media.get("animated"), list) else 0

    hashtags = t.get("hashtags", []) or []
    cashtags = t.get("cashtags", []) or []

    # inReplyToUser
    iru = t.get("inReplyToUser")
    inReplyToUser_val = iru.get("username") if isinstance(iru, dict) else iru

    # descriptionLinks y pinnedIds del user
    user_descriptionLinks_cnt = len(u.get("descriptionLinks", [])) if isinstance(u.get("descriptionLinks"), list) else 0
    user_pinnedIds_val = u.get("pinnedIds")
    if isinstance(user_pinnedIds_val, list):
        # lista de IDs → siempre string limpio
        user_pinnedIds_val = ",".join(str(int(float(x))) for x in user_pinnedIds_val if x is not None)
    elif user_pinnedIds_val is not None:
        # caso raro: valor único
        try:
            user_pinnedIds_val = str(int(float(user_pinnedIds_val)))
        except Exception:
            user_pinnedIds_val = str(user_pinnedIds_val)


    return {
        "id": get_id_str(t, "id"),
        "date": t.get("date"),

        "user": u.get("username"),
        "user_id": get_id_str(u, "id"),
        "user_displayname": u.get("displayname"),
        "user_rawDescription": u.get("rawDescription"),
        "user_created": u.get("created"),
        "user_followersCount": u.get("followersCount"),
        "user_friendsCount": u.get("friendsCount"),
        "user_statusesCount": u.get("statusesCount"),
        "user_favouritesCount": u.get("favouritesCount"),
        "user_listedCount": u.get("listedCount"),
        "user_mediaCount": u.get("mediaCount"),
        "user_location": u.get("location"),
        "user_protected": u.get("protected"),
        "user_verified": u.get("verified"),
        "user_blue": u.get("blue"),
        "user_blueType": u.get("blueType"),
        "user_descriptionLinks": user_descriptionLinks_cnt,
        "user_pinnedIds": user_pinnedIds_val,

        "lang": t.get("lang"),
        "rawContent": t.get("rawContent"),
        "replyCount": t.get("replyCount"),
        "retweetCount": t.get("retweetCount"),
        "likeCount": t.get("likeCount"),
        "quoteCount": t.get("quoteCount"),
        "bookmarkedCount": t.get("bookmarkedCount"),
        "conversationId": get_id_str(t, "conversationId"),

        "hashtags": ",".join(hashtags) if hashtags else None,
        "cashtags": ",".join(cashtags) if cashtags else None,
        "mentionedUsers_ids": ",".join(mentioned_ids) if mentioned_ids else None,
        "mentionedUsers_usernames": ",".join(mentioned_usernames) if mentioned_usernames else None,

        "viewCount": t.get("viewCount"),

        "quotedTweet_id": get_id_str(qt, "id") if qt else None,
        "quotedTweet_date": qt.get("date") if qt else None,
        "quotedTweet_user_id": get_id_str(qt_user, "id") if qt_user else None,
        "quotedTweet_user_username": qt_user.get("username") if qt_user else None,

        "place": json.dumps(t.get("place"), ensure_ascii=False) if isinstance(t.get("place"), (dict, list)) else t.get("place"),
        "inReplyToTweetId": get_id_str(t, "inReplyToTweetId"),
        "inReplyToUser": inReplyToUser_val,
        "sourceLabel": t.get("sourceLabel"),

        "media_photos_count": photos_cnt,
        "media_videos_count": videos_cnt,
        "media_animated_count": animated_cnt,
    }

# -------- lectura --------
src = Path("traficogt.txt")
rows = []
bad_lines = 0
with src.open("r", encoding="utf-16") as f:
    for i, ln in enumerate(f, 1):
        ln = ln.strip()
        if not ln:
            continue
        try:
            t = json.loads(ln)
            rows.append(flatten_tweet(t))
        except json.JSONDecodeError:
            bad_lines += 1

print(f"Filas parseadas: {len(rows)} | líneas malas: {bad_lines}")

df = pd.DataFrame(rows)

# Reordenar columnas
ordered_cols = [
    "id","date","user",
    "user_id","user_displayname","user_rawDescription","user_created",
    "user_followersCount","user_friendsCount","user_statusesCount","user_favouritesCount",
    "user_listedCount","user_mediaCount","user_location","user_protected","user_verified",
    "user_blue","user_blueType","user_descriptionLinks","user_pinnedIds",
    "lang","rawContent","replyCount","retweetCount","likeCount","quoteCount",
    "bookmarkedCount","conversationId",
    "hashtags","cashtags",
    "mentionedUsers_ids","mentionedUsers_usernames",
    "viewCount",
    "quotedTweet_id","quotedTweet_date","quotedTweet_user_id","quotedTweet_user_username",
    "place","inReplyToTweetId","inReplyToUser","sourceLabel",
    "media_photos_count","media_videos_count","media_animated_count",
]
df = df.reindex(columns=[c for c in ordered_cols if c in df.columns])

print("Dimensiones:", df.shape)
display(df.head(3))

# Guardar CSV en UTF-8
df.to_csv("traficogt_flat.csv", index=False, encoding="utf-8")
print("CSV escrito en: traficogt_flat.csv")


Filas parseadas: 5604 | líneas malas: 1
Dimensiones: (5604, 44)


,id,date,user,user_id,user_displayname,user_rawDescription,user_created,user_followersCount,user_friendsCount,user_statusesCount,...,quotedTweet_date,quotedTweet_user_id,quotedTweet_user_username,place,inReplyToTweetId,inReplyToUser,sourceLabel,media_photos_count,media_videos_count,media_animated_count
0,1834236045598056867,2024-09-12 14:22:06+00:00,traficogt,93938886,traficoGT,Noticias de ciudad de Guatemala,2009-12-01 20:42:19+00:00,314368,137,52385,...,2024-09-12 00:39:56+00:00,976875408,monymmorales,None,None,None,Twitter for Android,0,0,0
1,1834029142565658846,2024-09-12 00:39:56+00:00,monymmorales,976875408,Mony,Iglesia y estado son asunto separado.\nCatólic...,2012-11-28 20:16:36+00:00,5502,4887,180436,...,None,None,None,None,None,None,Twitter for Android,0,0,0
2,1834039491826180424,2024-09-12 01:21:04+00:00,animaldgalaccia,1730828822029750272,Jairo De La Nada,Sancarlista. Estudiante de filosofía. Ideológi...,2023-12-02 05:58:56+00:00,571,158,8875,...,None,None,None,None,None,PNCdeGuatemala,Twitter for iPhone,0,0,0


CSV escrito en: traficogt_flat.csv


## Extraer mensajes

In [11]:
import pandas as pd

# --- columnas de la subtabla de mensajes (centradas en el tweet en sí) ---
message_cols = [
    # Identificación básica
    "id", "date", "conversationId", "inReplyToTweetId", "inReplyToUser",
    # Información del usuario que publicó
    "user", "user_id",
    # Contenido
    "rawContent", "lang", "hashtags", "cashtags",
    "mentionedUsers_ids", "mentionedUsers_usernames",
    "media_photos_count", "media_videos_count", "media_animated_count",
    "place", "sourceLabel",
    # Interacción
    "replyCount", "retweetCount", "likeCount", "quoteCount",
    "bookmarkedCount", "viewCount",
    # Datos de la cita (si existe)
    "quotedTweet_id", "quotedTweet_date", "quotedTweet_user_id", "quotedTweet_user_username",
]

# Quedarnos solo con las que existan en df (por robustez)
message_cols = [c for c in message_cols if c in df.columns]

# Construir subtabla
messages = df.loc[:, message_cols].copy()

# Asegurar que IDs estén en dtype string y llenar NA
id_like_cols = [
    c for c in messages.columns
    if c.lower() in ("id", "conversationid", "inreplytotweetid", "quotedtweet_id", "quotedtweet_user_id", "user_id")
    or c.lower().endswith("_id")
    or c.lower().endswith("_ids")
]

for c in id_like_cols:
    if c in messages.columns:
        messages[c] = messages[c].astype("string").fillna("NA")

# Llenar todos los valores nulos con "NA"
messages = messages.fillna("NA")

print("Subtabla mensajes -> columnas:", len(messages.columns), "| filas:", len(messages))
display(messages.head(5))

# Guardados
messages.to_csv("traficogt_messages.csv", index=False, encoding="utf-8")
print("CSV escrito en: traficogt_messages.csv")


Subtabla mensajes -> columnas: 28 | filas: 5604


,id,date,conversationId,inReplyToTweetId,inReplyToUser,user,user_id,rawContent,lang,hashtags,...,replyCount,retweetCount,likeCount,quoteCount,bookmarkedCount,viewCount,quotedTweet_id,quotedTweet_date,quotedTweet_user_id,quotedTweet_user_username
0,1834236045598056867,2024-09-12 14:22:06+00:00,1834236045598056867,NA,NA,traficogt,93938886,Es comprensible la resolución... El ruso sabe ...,es,NA,...,0,0,1,0,0,393.0,1834029142565658846,2024-09-12 00:39:56+00:00,976875408,monymmorales
1,1834029142565658846,2024-09-12 00:39:56+00:00,1834029142565658846,NA,NA,monymmorales,976875408,La corrupción de la @CC_Guatemala\nes descarad...,es,NA,...,0,56,84,4,1,1587.0,NA,NA,NA,NA
2,1834039491826180424,2024-09-12 01:21:04+00:00,1834039491826180424,NA,PNCdeGuatemala,animaldgalaccia,1730828822029750272,@PNCdeGuatemala @mingobguate @FJimenezmingob @...,qme,NA,...,0,0,1,0,0,91.0,NA,NA,NA,NA
3,1833963729136091179,2024-09-11 20:20:01+00:00,1833963727575781534,1833963727575781534,EstacionDobleA,EstacionDobleA,1802661334355456000,@amilcarmontejo @AztecaNoticiaGT @BancadaSemil...,qam,NA,...,0,0,0,0,0,46.0,NA,NA,NA,NA
4,1833665391698092330,2024-09-11 00:34:31+00:00,1833664431084949711,1833664431084949711,CubReserva,CubReserva,1155617398675988481,@soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...,es,NA,...,0,0,1,0,0,171.0,NA,NA,NA,NA


CSV escrito en: traficogt_messages.csv


## preparacion

In [12]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Descargar stopwords si no las tienes
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

def clean_text(text):
    if pd.isna(text) or text == "NA":
        return "NA"
    
    # Convertir a string
    text = str(text)
    
    # Si es muy corto o vacío después de limpiar, devolver NA
    if len(text.strip()) <= 1:
        return "NA"
    
    # Convertir a minúsculas
    text = text.lower()
    
    # Quitar URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    
    # Quitar menciones (@usuario) pero conservar el texto
    text = re.sub(r"@\w+", "", text)
    
    # Quitar hashtags (#) pero conservar el texto
    text = re.sub(r"#(\w+)", r"\1", text)
    
    # Quitar caracteres especiales excepto letras, números y espacios básicos
    text = re.sub(r"[^\w\sáéíóúñü]", " ", text)
    
    # Quitar espacios extra
    text = re.sub(r"\s+", " ", text).strip()
    
    # Tokenizar y quitar stopwords
    tokens = [t for t in text.split() if t not in stopwords_es and len(t) > 2]
    
    if not tokens:
        return "NA"
    
    return " ".join(tokens)

# Aplicar sobre la columna rawContent (o la que quieras)
df["rawContent_clean"] = df["rawContent"].apply(clean_text)

# Llenar NA explícito
df["rawContent_clean"] = df["rawContent_clean"].fillna("NA")

# Vista rápida
display(df[["rawContent", "rawContent_clean"]].head(10))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,rawContent,rawContent_clean
0,Es comprensible la resolución... El ruso sabe ...,comprensible resolución ruso sabe engrasar maq...
1,La corrupción de la @CC_Guatemala\nes descarad...,corrupción descarada falsificación documentos ...
2,@PNCdeGuatemala @mingobguate @FJimenezmingob @...,NA
3,@amilcarmontejo @AztecaNoticiaGT @BancadaSemil...,NA
4,@soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...,urgente zona deterioro tala inmoderada tráfico...
5,@amilcarmontejo @PMTMuniGuate @Noti7Guatemala ...,avenidas calles avenida calle luz semáforo age...
6,Favor compartir \nEl vive el zona 7 Bethania ...,favor compartir vive zona bethania urgente des...
7,@traficogt @_ojoconmipisto @soy_502 @AztecaNot...,importante ponerle atención vecinos
8,@piero_coen @FJimenezmingob @traficogt @mingob...,pajas mota ahora vale
9,@erwin_fern84019 @piero_coen @FJimenezmingob @...,ayyyy arde fumen mota mejor segui chimando bebes
